# CMPD Car Theft Project

## By: Connor , Waqas, Issam , Rishabh
---




### Importing required libraries and data

In [18]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [19]:
# Loading the 2011-2016 CMPDData

# 2011 Incident Data
incident_2011 = pd.read_csv("2011_Incident.csv")

# 2012 Incident Data
incident_2012 = pd.read_csv("2012_Incident.csv")

# 2013 Incident Data
incident_2013 = pd.read_csv("2013_Incident.csv")

# 2014 Incident Data
incident_2014 = pd.read_csv("2014_Incident.csv")

# 2015 Incident Data
incident_2015 = pd.read_csv("2015_Incident.csv")

# 2016 Incident Data
incident_2016 = pd.read_csv("2016_Incident.csv")


## Exploratory Data Analysis (EDA)

Merging all CMPD Data (2011-2016)

In [20]:
# merging all the data sets into one
all_incident_df = pd.concat([incident_2011, incident_2012, incident_2013, incident_2014, incident_2015, incident_2016], ignore_index=True)


In [21]:
all_incident_df.head()

,Complaint_No,Block_No,Direction,Street_Name,Street_Type,Suffix,Apt_No,City,State,Zipcode,...,Clearance_Status,Clearance_Date,Case_Status,Reporting_Agency,Follow_up_Section,NIBRS_Hi_Class,Incident_From_Time,Incident_to_Time,Unnamed: 28,Unnamed: 29
0,20110101000308,4425,,EDDLEMAN,RD,,,CHARLOTTE,NC,28208,...,Normal Clearance - Cleared by Arrest ...,1/1/2011,Close/Cleared,Charlotte-Mecklenburg Police Department,27-FREEDOM DIVISION ...,Drug Equipment Violations ...,NaN,NaN,NaN,NaN
1,20110101000700,2228,,BEATTIES FORD,RD,,,CHARLOTTE,NC,28216,...,Normal Clearance - Cleared by Arrest ...,1/1/2011,Close/Cleared,Charlotte-Mecklenburg Police Department,02-METRO DIVISION ...,Aggravated Assault ...,NaN,NaN,NaN,NaN
2,20110101001104,2300,N,TRYON,ST,,,CHARLOTTE,NC,,...,Open ...,,Inactive,Charlotte-Mecklenburg Police Department,02-METRO DIVISION ...,Damage/Vandalism Of Property ...,NaN,NaN,NaN,NaN
3,20110101001302,4027,,QUAIL GLENN,CT,,K,CHARLOTTE,NC,28226,...,Open ...,,Inactive,Charlotte-Mecklenburg Police Department,22-SOUTH DIVISION ...,Burglary/B&E ...,NaN,NaN,NaN,NaN
4,20110101002401,150,S,COLLEGE,ST,,,CHARLOTTE,NC,28202,...,Normal Clearance - Cleared by Arrest ...,1/1/2011,Close/Cleared,Charlotte-Mecklenburg Police Department,01-CENTRAL DIVISION ...,Affray ...,NaN,NaN,NaN,NaN


In [22]:
all_incident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566789 entries, 0 to 566788
Data columns (total 30 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Complaint_No         566789 non-null  int64  
 1   Block_No             566789 non-null  object 
 2   Direction            566789 non-null  object 
 3   Street_Name          566789 non-null  object 
 4   Street_Type          566788 non-null  object 
 5   Suffix               566789 non-null  object 
 6   Apt_No               566789 non-null  object 
 7   City                 566789 non-null  object 
 8   State                566789 non-null  object 
 9   Zipcode              566789 non-null  object 
 10  X_Coordinate         566789 non-null  object 
 11  Y_Coordinate         566789 non-null  object 
 12  Division             566789 non-null  object 
 13  Reported_Date        566789 non-null  object 
 14  Location_Desc        566788 non-null  float64
 15  Location_Type    

In [23]:
print(all_incident_df.columns)

Index(['Complaint_No', 'Block_No', 'Direction', 'Street_Name', 'Street_Type',
       'Suffix', 'Apt_No', 'City', 'State', 'Zipcode', 'X_Coordinate',
       'Y_Coordinate', 'Division', 'Reported_Date', 'Location_Desc',
       'Location_Type', 'Incident_From_Date', 'Incident_to_date', 'Place1',
       'Place2', 'Clearance_Status', 'Clearance_Date', 'Case_Status',
       'Reporting_Agency', 'Follow_up_Section', 'NIBRS_Hi_Class',
       ' Incident_From_Time', 'Incident_to_Time', 'Unnamed: 28',
       'Unnamed: 29'],
      dtype='object')


We need to combine 'Direction', 'Street Name' and 'Street Type' into 1 column called 'Street Address'

In [24]:
# Combining Direction, Street_Name, Street_Type into Street_Address
all_incident_df.loc[:, 'Street_Address'] = all_incident_df['Direction'] + ' ' + all_incident_df['Street_Name'] + ' ' + all_incident_df['Street_Type']


We can get rid of all the extra columns that are not very relevant to our project.


In [25]:
# Adjusting the columns we want in our dataset
columns_to_keep = ['Street_Address','Zipcode','X_Coordinate','Y_Coordinate','NIBRS_Hi_Class']

all_incident_df = all_incident_df[columns_to_keep]

In [26]:
all_incident_df.head()

,Street_Address,Zipcode,X_Coordinate,Y_Coordinate,NIBRS_Hi_Class
0,EDDLEMAN RD,28208,1433698,551674,Drug Equipment Violations ...
1,BEATTIES FORD RD,28216,1446676,556631,Aggravated Assault ...
2,N TRYON ST,,1456850,549120,Damage/Vandalism Of Property ...
3,QUAIL GLENN CT,28226,1453488,502939,Burglary/B&E ...
4,S COLLEGE ST,28202,1449765,542246,Affray ...


In [27]:
# Displaying unique values in the 'NIBRS_Hi_Class' column
print(all_incident_df['NIBRS_Hi_Class'].unique())


['Drug Equipment Violations                                                                           '
 'Aggravated Assault                                                                                  '
 'Damage/Vandalism Of Property                                                                        '
 'Burglary/B&E                                                                                        '
 'Affray                                                                                              '
 'All Other Offenses                                                                                  '
 'Simple Assault                                                                                      '
 'Missing Person                                                                                      '
 'Drug/Narcotic Violations                                                                            '
 'Indecent Exposure                                             

In [28]:
# filtering by 'Motor Vehicle Theft'
all_incident_df['NIBRS_Hi_Class'] = all_incident_df['NIBRS_Hi_Class'].str.strip()
vehicle_df = all_incident_df[all_incident_df['NIBRS_Hi_Class'].str.contains('Motor Vehicle Theft', case=False)]

vehicle_df.head(15)

,Street_Address,Zipcode,X_Coordinate,Y_Coordinate,NIBRS_Hi_Class
84,INTERURBAN AV,28208,1431508,554267,Motor Vehicle Theft
89,SPRINGMONT LN,28208,1427405,557580,Motor Vehicle Theft
100,SCOTT FUTRELL DR,28208,1425917,546914,Motor Vehicle Theft
103,LISBON LN,28269,1463794,564750,Motor Vehicle Theft
110,PERIMETER STATION DR,28269,1447340,585770,Motor Vehicle Theft
113,JORDANS POND LN,28214,1423483,566774,Motor Vehicle Theft
148,E INDEPENDENCE BV,28212,1477862,522932,Motor Vehicle Theft
149,N CALDWELL ST,28202,1451678,542269,Motor Vehicle Theft
160,E 7 TH ST,,1451249,542901,Motor Vehicle Theft
165,MAGNOLIA HILL DR,28205,1470918,543000,Motor Vehicle Theft


We have some empty values under 'Zipcode' , 'X_Coordinate' , and 'Y_Coordinate' that we have to deal with:
 * Most of them are empty strings and whitespaces.

* Idealy we could use the X & Y coordinates to fill in the missing zip codes. Or fill in the missing information by using the street address. However this would take some time and we are on a bit of a time crunch, so we chose to remove rows entirely with missing data.

* Although this gives us less data to work with, we'd rather have accurate data that will work well with modeling than tons of information that will cause problems.




In [29]:
# Remove rows with NaN values
vehicle_df = vehicle_df.dropna()

# Remove rows with empty strings
vehicle_df = vehicle_df[~vehicle_df.apply(lambda row: row.str.strip().eq('').any(), axis=1)]

# Checking the cleaned DataFrame
print(vehicle_df)
vehicle_df.head(15)

# Creating a combined csv
vehicle_df.to_csv('Vehicle_Theft.csv', index=False)

                                        Street_Address Zipcode X_Coordinate  \
84         INTERURBAN                               AV   28208      1431508   
89         SPRINGMONT                               LN   28208      1427405   
100        SCOTT FUTRELL                            DR   28208      1425917   
103        LISBON                                   LN   28269      1463794   
110        PERIMETER STATION                        DR   28269      1447340   
...                                                ...     ...          ...   
566624     BEATTIES FORD                            RD   28216      1446074   
566626     NEVILLE ABBEY                            DR   28262      1483602   
566631     BREEZEWOOD                               DR   28262      1479544   
566668     BENNETTSVILLE                            LN   28262      1463389   
566755     HERRIN                                   AV   28205      1464791   

       Y_Coordinate       NIBRS_Hi_Class  
84      

In [33]:
# Group data by street address and calculate the count of vehicle theft incidents per street
street_theft_counts = vehicle_df['Street_Address'].value_counts().reset_index()
street_theft_counts.columns = ['Street_Address', 'Theft_Count']
zip_theft_counts = vehicle_df['Zipcode'].value_counts().reset_index()
zip_theft_counts.columns = ['Zipcode', 'Theft_Count']
# Displaying the counts for each address
print(street_theft_counts.head(15))
print(zip_theft_counts.head)



                                    Street_Address  Theft_Count
0   N  TRYON                                    ST          215
1   E  INDEPENDENCE                             BV          129
2      CENTRAL                                  AV          125
3      SOUTH                                    BV          113
4      ALBEMARLE                                RD          110
5      EASTWAY                                  DR           94
6      WILKINSON                                BV           93
7   S  TRYON                                    ST           76
8      MONROE                                   RD           64
9   N  DAVIDSON                                 ST           63
10     BEATTIES FORD                            RD           60
11     RENTAL CAR                               RD           58
12  W  SUGAR CREEK                              RD           52
13     THE PLAZA                                             52
14     WEST                             

## Data Enrichment
Still working on:

* Filtering and cleaning additional information about (neighborhood, demographics, etc.) Census data for years 2011 - 2016.

* Percent per capita crime prediction (Car theft)


## Machine Learning Model (In Progress)

### Splitting the data

The features we plan on using may change depending on if we can prepare the additional data correctly. However our target varialbe should remain the same ('Theft_Count').

for X = street_theft_counts[['', '', ...]]
it is a fill in the blank for when demographic data gets involved

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Features (Enriched data)
X = street_theft_counts[['', '', ...]]

# Target variable
y = street_theft_counts['Theft_Count']

In [36]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [38]:
# Building a RandomForestClassifier
# Building a Gradient Boosting Classifier

# clf = RandomForestClassifier(random_state=42)
# clf.fit(X_train, y_train)

from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train, y_train)

ValueError: could not convert string to float: '   HUBBARD WOODS                            RD'

In [ ]:
# Make predictions and evaluate the model

from sklearn.metrics import mean_absolute_error, mean_squarred_error

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [ ]:
# Print accuracy and classification report
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Accuracy: 0.8188868613138686
Classification Report:
              precision    recall  f1-score   support

       22821       0.00      0.00      0.00         0
       26216       0.00      0.00      0.00         1
       28020       0.00      0.00      0.00         1
       28034       0.00      0.00      0.00         1
       28078       1.00      0.50      0.67         2
       28105       0.78      0.70      0.74        10
       28110       0.00      0.00      0.00         1
       28112       0.00      0.00      0.00         1
       28115       0.00      0.00      0.00         1
       28134       1.00      0.80      0.89         5
282-9              0.00      0.00      0.00         0
        2820       0.00      0.00      0.00         3
       28201       1.00      0.33      0.50         3
       28202       0.83      0.88      0.85        96
       28203       0.77      0.93      0.84        40
       28204       0.75      0.75      0.75        12
       28205       0.94      